In [52]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id, format_number
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import FMRegressor
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.ml import Pipeline

In [53]:
# Создание SparkSession
spark = SparkSession.builder.appName("CrabAgePrediction").getOrCreate()
# Загружаем датасет
data = spark.read.csv("CrabAgePrediction.csv", header = True, inferSchema = True)

data.printSchema()
data.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)
 |-- Age: integer (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.6

In [54]:
# Преобразуем Пол краба в числа
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
# Преобразовываем данные. Начинаем с выбора признаков
features = ["SexIndex", "Length", "Diameter" , "Height", "Weight", "Shucked Weight", "Viscera Weight", "Shell Weight"]
# Создаем вектор этих признаков
assembler = VectorAssembler(inputCols=features, outputCol="features")
# Разделяем данные на обучение и тест
(train, test) = data.randomSplit([0.7, 0.3])
# Создаем кортеж среднеквадратичных ошибок
tupleRMSE = ()
Pred_arr = []

train.show(3)
test.show(3)

print(train.count())
print(test.count())

+---+------+--------+------+---------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|   Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+---------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|  2.26796|     0.8788345|    0.60951425|   0.7087375|  5|
|  F| 0.725|   0.525|0.1875|7.7961125|     3.2034935|    1.91359125|   0.9922325|  6|
|  F| 0.725|  0.5625|0.1875|  3.96893|    1.45999925|    0.66621325|     1.13398|  5|
+---+------+--------+------+---------+--------------+--------------+------------+---+
only showing top 3 rows

+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F|0.7625|  0.5625| 0.175|4.20990075|    1.65844575|    0.94970825|   1.2757275|  7|
|  F|  0.85|  0.6375|0.21

In [55]:
# Создаем модель
rf = RandomForestRegressor(featuresCol = "features", labelCol= "Age")
# Создаем конвейер
pipelineRF = Pipeline(stages = [indexer, assembler, rf])
# Обучение модели
modelRF = pipelineRF.fit(train)
# Предсказание возраста на тестовой выборке
predictionsRF = modelRF.transform(test).withColumnRenamed("prediction", "predictionsRF")
Pred_arr.append(predictionsRF)

In [56]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsRF", metricName = "rmse")
rmseRF = evaluator.evaluate(predictionsRF)
tupleRMSE += (rmseRF,)
print("Root Mean Squared Error (RMSE): %s" % rmseRF)
# Вывод результата и выключение SparkSession
predictionsRF.select("Age", "predictionsRF").show()

Root Mean Squared Error (RMSE): 2.4179375528537674
+---+------------------+
|Age|     predictionsRF|
+---+------------------+
|  7| 6.594659427450752|
|  6| 7.871283734425289|
|  6| 7.963288136657253|
| 10| 8.057965445331728|
|  8| 7.889020916800051|
|  5|7.9949800455895454|
| 10| 8.462476246929658|
|  7| 8.160344699210814|
|  7| 9.068308795603793|
| 10| 8.462476246929658|
| 10|  8.29551087009093|
| 10| 8.393243663180488|
| 13| 8.233380410131641|
| 16| 9.355811895018638|
|  9| 8.486886903117275|
|  9| 9.298196463508672|
| 10| 9.318517656294933|
| 12|10.424962074373818|
|  8| 9.425166021325294|
| 11|  9.19667671496469|
+---+------------------+
only showing top 20 rows



In [57]:
# Создание модели градиентного бустинга
gbt = GBTRegressor(featuresCol="features", labelCol= "Age")
# Создание конвейера
pipelineGBT = Pipeline(stages = [indexer, assembler, gbt])
# Обучение модели
modelGBT = pipelineGBT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGBT = modelGBT.transform(test).withColumnRenamed("prediction", "predictionsGBT")
Pred_arr.append(predictionsGBT)

In [58]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGBT", metricName = "rmse")
rmseGBT = evaluator.evaluate(predictionsGBT)
tupleRMSE += (rmseGBT,)
print("Root Mean Squared Error (RMSE): %s" % rmseGBT)
# Вывод результата и выключение SparkSession
predictionsGBT.select("Age", "predictionsGBT").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.4145932340710274
+---+------------------+
|Age|    predictionsGBT|
+---+------------------+
|  7|5.9472047948118885|
|  6| 9.357288828425332|
|  6| 8.040487717731033|
| 10| 8.230610220671146|
|  8| 8.040487717731033|
|  5| 9.342250036509883|
| 10| 8.634327184614628|
|  7| 9.306911677340077|
|  7|  9.62479973457039|
| 10| 8.634327184614628|
| 10| 8.233503778090963|
| 10|  8.25184284129502|
| 13|  8.45038482478648|
| 16| 9.475987231917298|
|  9| 9.036663026314256|
|  9| 9.606460671366333|
| 10| 9.340924507966234|
| 12|11.771595597993523|
|  8| 9.449261409932369|
| 11| 9.482448761915451|
+---+------------------+
only showing top 20 rows



In [59]:
# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="Age", maxIter=10, regParam=0.3, elasticNetParam=0.8) 
# Создание конвейера
pipelineLR = Pipeline(stages = [indexer, assembler, lr])
# Fit the model
modelLR = pipelineLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsLR = modelLR.transform(test).withColumnRenamed("prediction", "predictionsLR")
Pred_arr.append(predictionsLR)

In [60]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsLR", metricName = "rmse")
rmseLR = evaluator.evaluate(predictionsLR)
tupleRMSE += (rmseLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseLR)
# Вывод результата и выключение SparkSession
predictionsLR.select("Age", "predictionsLR").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.6814091966136013
+---+-----------------+
|Age|    predictionsLR|
+---+-----------------+
|  7|7.312727958505367|
|  6| 7.50380084774897|
|  6|7.582345492675976|
| 10| 7.80407872046244|
|  8|7.648862659430991|
|  5|7.722683761183636|
| 10|7.955506264365743|
|  7|7.802403082134456|
|  7| 8.25148922417507|
| 10|8.029327366118387|
| 10| 8.14068093441117|
| 10|8.079792064325801|
| 13|8.020150202298197|
| 16|8.395693775316655|
|  9|8.122248841265012|
|  9|8.385968156237446|
| 10| 8.80486464948844|
| 12| 8.82138463105915|
|  8| 8.64086117215413|
| 11|8.442292190890145|
+---+-----------------+
only showing top 20 rows



In [61]:
# Создание модели решающих деервьев
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineDT = Pipeline(stages = [indexer, assembler, dt])
# Fit the model
modelDT = pipelineDT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsDT = modelDT.transform(test).withColumnRenamed("prediction", "predictionsDT")
Pred_arr.append(predictionsDT)

In [62]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsDT", metricName = "rmse")
rmseDT = evaluator.evaluate(predictionsDT)
tupleRMSE += (rmseDT,)
print("Root Mean Squared Error (RMSE): %s" % rmseDT)
# Вывод результата и выключение SparkSession
predictionsDT.select("Age", "predictionsDT").show()

Root Mean Squared Error (RMSE): 2.460936088145193
+---+------------------+
|Age|     predictionsDT|
+---+------------------+
|  7| 5.942857142857143|
|  6| 9.352941176470589|
|  6| 8.297872340425531|
| 10| 8.297872340425531|
|  8| 8.297872340425531|
|  5| 9.352941176470589|
| 10| 8.297872340425531|
|  7| 9.352941176470589|
|  7| 9.502958579881657|
| 10| 8.297872340425531|
| 10| 8.297872340425531|
| 10| 8.297872340425531|
| 13| 8.297872340425531|
| 16| 9.502958579881657|
|  9| 9.502958579881657|
|  9| 9.502958579881657|
| 10| 9.502958579881657|
| 12|10.805555555555555|
|  8| 9.502958579881657|
| 11| 9.502958579881657|
+---+------------------+
only showing top 20 rows



In [63]:
# Создание модели изотонической регрессии
ir = IsotonicRegression(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineIR = Pipeline(stages = [indexer, assembler, ir])
# Fit the model
modelIR = pipelineIR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsIR = modelIR.transform(test).withColumnRenamed("prediction", "predictionsIR")
Pred_arr.append(predictionsIR)

In [64]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsIR", metricName = "rmse")
rmseIR = evaluator.evaluate(predictionsIR)
tupleRMSE += (rmseIR,)
print("Root Mean Squared Error (RMSE): %s" % rmseIR)
# Вывод результата и выключение SparkSession
predictionsIR.select("Age", "predictionsIR").show()

Root Mean Squared Error (RMSE): 3.624850074638497
+---+-------------+
|Age|predictionsIR|
+---+-------------+
|  7|         11.0|
|  6|         11.0|
|  6|         11.0|
| 10|         11.0|
|  8|         11.0|
|  5|         11.0|
| 10|         11.0|
|  7|         11.0|
|  7|         11.0|
| 10|         11.0|
| 10|         11.0|
| 10|         11.0|
| 13|         11.0|
| 16|         11.0|
|  9|         11.0|
|  9|         11.0|
| 10|         11.0|
| 12|         11.0|
|  8|         11.0|
| 11|         11.0|
+---+-------------+
only showing top 20 rows



In [65]:
# Создание модели FMRegressor
fm = FMRegressor(featuresCol="features", labelCol="Age", predictionCol="prediction",stepSize=0.001)
# Создание конвейера
pipelineFM = Pipeline(stages = [indexer, assembler, fm])
# Fit the model
modelFM = pipelineFM.fit(train)
# Предсказание возраста на тестовой выборке
predictionsFM = modelFM.transform(test).withColumnRenamed("prediction", "predictionsFM")
Pred_arr.append(predictionsFM)

In [66]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsFM", metricName = "rmse")
rmseFM = evaluator.evaluate(predictionsFM)
tupleRMSE += (rmseFM,)
print("Root Mean Squared Error (RMSE): %s" % rmseFM)
# Вывод результата и выключение SparkSession
predictionsFM.select("Age", "predictionsFM").show()

Root Mean Squared Error (RMSE): 5.522154437259932
+---+------------------+
|Age|     predictionsFM|
+---+------------------+
|  7|1.1512895173329414|
|  6|1.4204368081670415|
|  6|1.5146357738547231|
| 10|1.5580293055296135|
|  8|1.5555683536719411|
|  5|1.7565467800276313|
| 10|1.7196058607925047|
|  7| 2.095855550549667|
|  7|2.1803587004735783|
| 10|1.8738783332995934|
| 10| 2.204154807017155|
| 10|2.0909657159703077|
| 13|1.9692696875090598|
| 16|2.5443563706573045|
|  9| 2.516111476156957|
|  9|2.5272427484656736|
| 10|  3.38541928675172|
| 12|2.8357197296195533|
|  8|2.9419795426335464|
| 11| 2.617531685228274|
+---+------------------+
only showing top 20 rows



In [67]:
# Модель  `AFTSurvivalRegression`  не подходит для датасета  `CrabAgePrediction`, так как нет информации о том,  
# живы ли были крабы, когда их поймали.

# Эта модель предназначена для анализа данных выживания, где есть данные о времени до события (в данном случае, смерть) 
# и информация о том,  было ли это событие наблюдено (краб умер) или  **цензурировано**  (краб был жив, когда его поймали).  

In [68]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="Age", family="gaussian", link="identity", maxIter=10, regParam=0.3)
# Создание конвейера
pipelineGLR = Pipeline(stages = [indexer, assembler, glr])
# Fit the model
modelGLR = pipelineGLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGLR = modelGLR.transform(test).withColumnRenamed("prediction", "predictionsGLR")
Pred_arr.append(predictionsGLR)

In [69]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGLR", metricName = "rmse")
rmseGLR = evaluator.evaluate(predictionsGLR)
tupleRMSE += (rmseGLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseGLR)
# Вывод результата и выключение SparkSession
predictionsGLR.select("Age", "predictionsGLR").show()

Root Mean Squared Error (RMSE): 2.480306054560324
+---+------------------+
|Age|    predictionsGLR|
+---+------------------+
|  7|6.7257143482142645|
|  6| 7.012450624758874|
|  6| 7.212301877671122|
| 10|  7.52178978718794|
|  8| 7.399663489061817|
|  5| 7.408322737372648|
| 10|  7.81052590807068|
|  7| 7.373699192238221|
|  7| 8.089792653320917|
| 10| 7.945737139683034|
| 10| 7.955860666767313|
| 10|  8.02361117859682|
| 13|  7.94769223533339|
| 16|  8.46245851371426|
|  9| 7.932500096802977|
|  9| 8.433056195117587|
| 10| 8.768477266251686|
| 12| 8.988716485388899|
|  8| 8.671714913814188|
| 11| 8.717477465388704|
+---+------------------+
only showing top 20 rows



In [75]:
# Создаем списки моделей, выполняющихся том же в порядке выше
modelsList = ["RandomForestRegressor", "GBTRegressor", "LinearRegression", "DecisionTreeRegressor", 
              "IsotonicRegression", "FMRegressor", "GeneralizedLinearRegression"]
shortModelsList = ["predictionsRF", "predictionsGBT", "predictionsLR", "predictionsDT", "predictionsIR", 
                   "predictionsFM", "predictionsGLR"]

# Схема нового DataFrame значений RMSE каждой модели
schemaRMSE = StructType([
    StructField(modelsList[0], DoubleType(), True),
    StructField(modelsList[1], DoubleType(), True),
    StructField(modelsList[2], DoubleType(), True),
    StructField(modelsList[3], DoubleType(), True),
    StructField(modelsList[4], DoubleType(), True),
    StructField(modelsList[5], DoubleType(), True),
    StructField(modelsList[6], DoubleType(), True)
])
# Создание DataFrame
rmseDF = spark.createDataFrame([tupleRMSE], schemaRMSE)
rmseDF.printSchema()

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)



In [76]:
for i, df in enumerate(Pred_arr):
    df = df.withColumn("id", monotonically_increasing_id())
    Pred_arr[i] = df

ListSelectedDF = [df.select("Age", "id", shortModelsList[i]) for i, df in enumerate(Pred_arr)]
predDF = ListSelectedDF[0]

for i in range(1, len(ListSelectedDF)):
    predDF = predDF.join(ListSelectedDF[i], on = ["Age", "id"], how = "inner")

predDF = predDF.drop("id")

predDF.printSchema()
rmseDF.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- predictionsRF: double (nullable = false)
 |-- predictionsGBT: double (nullable = false)
 |-- predictionsLR: double (nullable = false)
 |-- predictionsDT: double (nullable = false)
 |-- predictionsIR: double (nullable = false)
 |-- predictionsFM: double (nullable = false)
 |-- predictionsGLR: double (nullable = false)

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)



In [78]:
predDF.printSchema()
rmseDF.printSchema()
# Вывод лучшего результата
print(f"Лучшая модель: {modelsList[rmseDF.head().index(min(rmseDF.head()))]} (RMSE: {min(rmseDF.head())})")
# Запись результатов в csv-файл
rmseDF.write.csv("RMSE.csv", mode = "overwrite", header = True)
predDF.write.csv("Pred.csv", mode = "overwrite", header = True)

root
 |-- Age: integer (nullable = true)
 |-- predictionsRF: double (nullable = false)
 |-- predictionsGBT: double (nullable = false)
 |-- predictionsLR: double (nullable = false)
 |-- predictionsDT: double (nullable = false)
 |-- predictionsIR: double (nullable = false)
 |-- predictionsFM: double (nullable = false)
 |-- predictionsGLR: double (nullable = false)

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)

Лучшая модель: GBTRegressor (RMSE: 2.4145932340710274)


In [ ]:
spark.stop()